<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">

### License

<p style="font-family: 'Fira Code', monospace; font-size: 1.2rem">
Copyright (C) 2021-2022, Xilinx, Inc.<br>
Copyright (C) 2022-2025, Advanced Micro Devices, Inc.
<br><br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br><br>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>
</p>


# ChipScoPy DDR Reporting Example


<img src="../img/api_overview.png" width="500" align="left">

## Description
This demo shows how to print and report DDR calibration status and report detailed information.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2025.2 installed and running
- Xilinx cs_server 2025.2 installed and running
- Python 3.10 or greater installed
- ChipScoPy 2025.2 installed
- Jupyter notebook support and extra libs needed - Please do so, using the command `pip install chipscopy[core-addons]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- Paths to server(s) and files are set correctly

In [1]:
import pprint
import os
import json
from chipscopy import create_session, report_versions, delete_session
from chipscopy import get_design_files

In [2]:
# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# specify hw and if programming is desired
HW_PLATFORM = os.getenv("HW_PLATFORM", "vck190")
PROG_DEVICE = os.getenv("PROG_DEVICE", 'True').lower() in ('true', '1', 't')

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files(f"{HW_PLATFORM}/production/chipscopy_ced")

PDI_FILE = design_files.programming_file
LTX_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PDI_FILE}")
print(f"PROBES_FILE:{LTX_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: /scratch/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.pdi
PROBES_FILE:/scratch/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
# Start of the connection
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

                      ChipScoPy Version Information                      
┌────────────────────────────┬──────────────────────────────────────────┐
│                     Entity │ Version                                  │
├────────────────────────────┼──────────────────────────────────────────┤
│                  ChipScoPy │ ┌───────────┬──────────────────────┐     │
│                            │ │ Attribute │ Value                │     │
│                            │ ├───────────┼──────────────────────┤     │
│                            │ │     Build │ 2025.2.0.0           │     │
│                            │ │ Timestamp │ Dec 31 1969-16:00:00 │     │
│                            │ └───────────┴──────────────────────┘     │
│                            │                                          │
│ hw_server @ localhost:3121 │ ┌───────────┬───────────┐                │
│                            │ │ Attribute │ Value     │                │
│                            │ ├───────────┼───────────┤                │
│                            │ │    Status │ Connected │                │
│                            │ │     Build │ 0         │                │
│                            │ │   Version │ 2025.2    │                │
│                            │ │ Timestamp │           │                │
│                            │ └───────────┴───────────┘                │
│                            │                                          │
│ cs_server @ localhost:3042 │ ┌───────────────┬──────────────────────┐ │
│                            │ │     Attribute │ Value                │ │
│                            │ ├───────────────┼──────────────────────┤ │
│                            │ │        Status │ Connected            │ │
│                            │ │         Build │ 1762552890           │ │
│                            │ │       Version │ 2025.2.0             │ │
│                            │ │     Timestamp │ Nov 07 2025-14:01:30 │ │
│                            │ │       Package │ 2025.2.1762552890    │ │
│                            │ │ Artifact type │ pyinstaller          │ │
│                            │ └───────────────┴──────────────────────┘ │
└────────────────────────────┴──────────────────────────────────────────┘

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
versal_device = session.devices.filter_by(family="versal").get()
if PROG_DEVICE:
    versal_device.program(PDI_FILE)
else:
    print("skipping programming")

--> INFO: Programming device with: 
/scratch/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.pdi

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use

In [5]:
versal_device.discover_and_setup_cores()
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


## 5 - Show enabled DDRs in the device. Pick one to use

In [6]:
ddr_list = versal_device.ddrs
for ddr in ddr_list:
    print(ddr, "  Enabled:", ddr.is_enabled)

# Grab the first enabled DDR
for ddr in ddr_list:
    if ddr.is_enabled:
        print("Using Enabled DDR: ", ddr)
        break

ddr_0   Enabled: True
ddr_1   Enabled: False
ddr_2   Enabled: False
ddr_3   Enabled: False
Using Enabled DDR:  ddr_0


## 6 - Getting the Calibration Status

There are several methods available to collect memory calibration information.

### Method 1 - Calibration PASS/FAIL status

In [7]:
# Method 1 - Use Status string base API directly
print(ddr, "calibration status:", ddr.get_cal_status())

ddr_0 calibration status: PASS


### Method 2 - Calibration from the status property group

In [8]:
# Use Property Group to get dictionary base of results
props = ddr.ddr_node.get_property_group(["status"])
print(pprint.pformat(props, indent=2))

{ 'cal_error_msg': 'None',
  'cal_message': 'No errors detected during calibration.',
  'cal_status': 'PASS',
  'dqs_status': 'Running',
  'health_status': 'GOOD',
  'track_enable': 'Post-Cal Status, DDRMC ISR Tracking, UB ISR Tracking, DQS '
                  'Gate Tracking'}


### Method 3 - Detailed calibration status for each stage

In [9]:
# Use get Cal Stages API directly to also get dictionary results
props = ddr.get_cal_stages()
print(pprint.pformat(sorted(props.items()), indent=2))

[ ('cal_stage.01_F0_PHY_BISC', 'Pass'),
  ('cal_stage.02_F0_MEM_INIT', 'Pass'),
  ('cal_stage.03_F0_DQS_GATE_CAL', 'Pass'),
  ('cal_stage.04_F0_WRITE_LEVELING', 'Pass'),
  ('cal_stage.05_F0_READ_DQ_CAL', 'Pass'),
  ('cal_stage.06_F0_WRITE_DQ_DBI_CAL', 'Pass'),
  ('cal_stage.07_F0_WRITE_LATENCY_CAL', 'Pass'),
  ('cal_stage.08_F0_READ_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.09_F0_WRITE_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.10_EN_VT_TRACK', 'Pass'),
  ('cal_stage.11_READ_DQS_TRACK', 'Pass'),
  ('cal_stage.12_CAL_DONE', 'Pass')]


## 7 - Generate Full DDRMC Report

The report() API call creates a full DDRMC status report to stdout or a file. This report includes memory configuration, margin analysis, calibration, and health status information.

In [10]:
# Use a single report command to get all latest essential
# Status and decoded data collected as it presents
ddr.report()
# Specify True to argument 1, and name/path to argument 2
# to get the report output generated and saved to a file
ddr.report(True, "test_out.txt")
print("Report Done.\n")

-------------------

DDRMC Status

-------------------

Calibration Status:   PASS

Overall Health:   GOOD

...

Byte  4   -  Left Margin:   145  ( 74 )  Center Point:   563  ( 286 )  Right Margin:   112  ( 57 )

Byte  5   -  Left Margin:   139  ( 73 )  Center Point:   614  ( 320 )  Right Margin:   124  ( 65 )

Byte  6   -  Left Margin:   142  ( 74 )  Center Point:   610  ( 316 )  Right Margin:   123  ( 64 )

Byte  7   -  Left Margin:   145  ( 74 )  Center Point:   628  ( 320 )  Right Margin:   127  ( 65 )

Report Done.



## 8 - Dump the complete set of internal properties as json

This demonstrates how to get a Python dictionary of all the low level DDR properties. These can be converted to JSON easily for export to other tools.

In [11]:
props = ddr.ddr_node.get_property_group([])
json_props = json.dumps(props, indent=4)
print(json_props)

{
    "phy_rden10_dly": 22,
    "f1_wrlvl_fine_left_rank0_byte1": 0,
    "ddrmc_isr_nsu_fatal0": 0,
    "f0_dqsgate_stg1_overflow_18": 0,
    "f0_wrdqdbi_odly_dq_final_bit49": 50,
    "f0_rdcmplx_nqtr_right_nibble07": 146,
    "f1_rddq_idelay_final_bit40": 0,
    "f1_rddq_nqtr_right_fcrse_nibble01": 0,
    "cal_seq_status_05": 263,
    "f0_rd_comp_rise_ctp_nib_05": [
        89,
        174
    ],
    "cal_seq_status_01": 385,
    "f0_wrdqdbi_deskew_dq_odly_fcrse_bit45": 130,
    "f1_rddq_idelay_final_bit38": 0,
    "wrvref_vref_low_bound_rank0_byte4": 0,
    "iobmap_dq_41_40": 0,
    "f0_dqsgate_stg1_overflow_17": 0,
    "f1_wrdqdbi_stg2_dq_odly_bit34": 0,
    "cal_status": "PASS",
    "bisc_idly4_align_nibble10": 74,
    "f0_rddq_pqtr_left_nibble01": 17,
    "f1_rddq_nqtr_left_nibble10": 0,
    "cal_seq_status_11": 397,
    "f0_wrdqdbi_stg3_dq_odly_bit18": 29,
    "cal_error_phy_nibble_8_0": 0,
    "cal_stage_time_33": 0,
    "f1_wrlvl_offset_rank0_byte7": 0,
    "f1_wrlvl_fine_final

In [12]:
## When done with testing, close the connection
delete_session(session)